In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
from collections import Counter

In [3]:
import nltk

In [105]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tron/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [109]:
val = pd.read_pickle('pre_data.pkl')

In [24]:
val.shape

(121512, 3)

In [22]:
val.head()

,image_id,question,answer
0,350623,What is the table made of?,wood
1,350623,Is the food napping on the table?,no
2,350623,What has been upcycled to make lights?,kettles
3,8647,Is this an Spanish town?,no
4,8647,Are there shadows on the sidewalk?,yes


In [10]:
def get_ans_vocab(data,col_name,max_val = 1000):
    import operator
    from collections import defaultdict
    freq = defaultdict(int)
    b=10
    for line in (data[col_name].unique()):
        for word in nltk.word_tokenize(str(line)):
            freq[word.lower()]+=1

    sort_freq = sorted(freq.items(),key=operator.itemgetter(1),reverse=True)[0:max_val]
    top_answers, top_fq = zip(*sort_freq)
    return top_answers

In [11]:
# def get_ques_vocab():
ques = get_ans_vocab(val,'question')

In [12]:
ans = get_ans_vocab(val, 'answer')

# Model designing

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Flatten, Embedding, Merge

In [ ]:
left_vgg = Sequential()
left_vgg.add(Dense(300, input_dim=4096, activation='relu'))

centre_w2vec = Sequential()
embedding_layer = Embedding(len(word_index) + 1,embed_dim,weights=[embedding_matrix],input_length=max_seq,trainable=False)
#centre_w2vec.add(Dense(500,input_dim=300))

right_vgg = Sequential()
right_vgg.add(Dense(300, input_dim=4096, activation='relu'))

merge_layer = Merge([left_vgg,centre_w2vec,right_vgg], mode='concat')

lstm_model = Sequential()
lstm_model.add(merge_layer)
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(LSTM(1000, input_shape=(1+max_seq+1,300)))
lstm_model.add(Dense(1000, activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
lstm_model.fit([input_data_1, input_data_2, input_data_3], targets)

# Encoding



In [7]:
val.columns

Index([u'image_id', u'question', u'answer'], dtype='object')

In [13]:
ques_list = list(val.question.values)

In [22]:
# ques_list

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [53]:
from keras.preprocessing.text import base_filter
from keras.utils import np_utils

In [81]:
max_word = 2000
max_seq = 20

In [45]:
d = [s.encode('ascii') for s in ques_list]

In [50]:
print ques_list[1:5]

[u'Is the food napping on the table?', u'What has been upcycled to make lights?', u'Is this an Spanish town?', u'Are there shadows on the sidewalk?']


In [82]:
tokenizer = Tokenizer(nb_words=max_word, filters=base_filter(), lower=True, split=" ")
tokenizer.fit_on_texts(d)
sequences = tokenizer.texts_to_sequences(d)

word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=max_seq)
# labels = np_utils.to_categorical(np.asarray())
print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)

('Shape of data tensor:', (121512, 20))


In [102]:
# a = np.zeros(shape=(2,2))
# print a
# a[0] = np.array([1,1])
# print a

In [78]:
word_index['napping']

3175

In [91]:
data[1,:]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  1, 49,
        7,  1, 67], dtype=int32)

In [84]:
data.shape[0]

121512

In [106]:
# print word_index.get

# Embedded matrix

In [104]:
import pickle
from keras.layers import Embedding

In [ ]:
embed_dim = 300

with open('embeddings/embedding_matrix','r') as f:
    embedding = pickle.load(file)
with open('embeddings/word_idx','r') as f:
    word_idx = pickle.load(file)

embedding_matrix = np.zeros(shape=(len(word_index)+1), embed_dim)
for word,freq in word_index.items():
    embedding_matrix[freq] = embedding[word_idx[word]]


In [ ]:
embedding_layer = Embedding(len(word_index) + 1,embed_dim,weights=[embedding_matrix],input_length=max_seq,trainable=False)

# VGG16/19 net loader

In [107]:
import numpy as np
import pandas as pd
import scipy as sc

def get_image_labels(pickle_file_path):
    data_frame = pd.read_pickle(pickle_file_path)
    labels = data_frame[['image_id']].values
    return labels

def get_image_features(img_ids,vgg_model_path):
    features_struct = sc.io.loadmat(vgg_model_path)
    VGGfeatures = features_struct['feats']
    id_map = {}
    for ids in img_ids:
        ids_split = ids.split()
        id_map[id_split[0]] = int(id_split[1])
    nb_samples = len(img_ids)
    nb_dimensions = VGGfeatures.shape[0]
    image_matrix = np.zeros((nb_samples, nb_dimensions))
    for j in range(nb_samples):
        image_matrix[j,:] = VGGfeatures[:,id_map[img_ids[j]]]
    return image_matrix

In [116]:
img_ids = val.image_id.values.tolist()

In [117]:
img_ids[1]

350623